In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC5/Dendritic/Kendall/Pairs.tsv.gz"
path.pairs.ABC = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC5/Dendritic/Predictions/EnhancerPredictionsAllPutative.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC5/Dendritic/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/3.merge_cell_type.240522/RNA_count_matrix/Dendritic.rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.Dendritic.240524/"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ABC results

In [5]:
pairs.ABC = readGeneric(path.pairs.ABC,
                        header = T,
                        keep.all.metadata = T)

Filter ABC results which distance < 1M

In [6]:
pairs.ABC.1M = pairs.ABC[pairs.ABC$distance < 10^6]

Filter E-G pairs overlaping with pairs.ABC.1M

In [7]:
df.pairs.E2G.chr_rename = as.data.frame(pairs.E2G)[,1:3]
df.pairs.E2G.chr_rename[,"seqnames"] = paste(seqnames(pairs.E2G),
                                             mcols(pairs.E2G)[,"TargetGene"],
                                             sep = "_")
pairs.E2G.chr_rename = GRanges(df.pairs.E2G.chr_rename)
rm(df.pairs.E2G.chr_rename)

df.pairs.ABC.1M.chr_rename = as.data.frame(pairs.ABC.1M)[,1:3]
df.pairs.ABC.1M.chr_rename[,"seqnames"] = paste(seqnames(pairs.ABC.1M),
                                                mcols(pairs.ABC.1M)[,"TargetGene"],
                                                sep = "_")
pairs.ABC.1M.chr_rename = GRanges(df.pairs.ABC.1M.chr_rename)
rm(df.pairs.ABC.1M.chr_rename)

pairs.E2G.filter = pairs.E2G[countOverlaps(pairs.E2G.chr_rename,
                                           pairs.ABC.1M.chr_rename) > 0]
rm(pairs.E2G.chr_rename)
rm(pairs.ABC.1M.chr_rename)
pairs.E2G.filter

GRanges object with 2894315 ranges and 3 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100015450-100016026      * |         AGL
        [2]     chr1 100015450-100016026      * |      CDC14A
        [3]     chr1 100015450-100016026      * |         DBT
        [4]     chr1 100015450-100016026      * |       EXTL2
        [5]     chr1 100015450-100016026      * |     MFSD14A
        ...      ...                 ...    ... .         ...
  [2894311]     chrX     9995660-9996208      * |       TBL1X
  [2894312]     chrX     9995660-9996208      * |        WWC3
  [2894313]     chrX     9997579-9998101      * |       CLCN4
  [2894314]     chrX     9997579-9998101      * |       TBL1X
  [2894315]     chrX     9997579-9998101      * |        WWC3
                          PeakName               PairName
                       <character>            <character>
        [1] chr1-10

Import ATAC matrix

In [8]:
matrix.atac = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [9]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

In [10]:
pairs.E2G.filter2 = pairs.E2G.filter[pairs.E2G.filter$TargetGene %in% rownames(matrix.rna) &
                                     pairs.E2G.filter$PeakName %in% rownames(matrix.atac)]

Prepare SCENT meta data

In [11]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = "K562")
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
s1d1_CGGGCTTAGTTGGGCC-1,s1d1_CGGGCTTAGTTGGGCC-1,4108,69,K562,8.320692,0.016796495
s1d1_GCGCCTTGTGCATTAG-1,s1d1_GCGCCTTGTGCATTAG-1,4749,99,K562,8.465689,0.020846494
s1d1_AACCGGCTCAACCAAC-1,s1d1_AACCGGCTCAACCAAC-1,3337,42,K562,8.112827,0.012586155
s1d1_GCTCTGTTCTTGTCTG-1,s1d1_GCTCTGTTCTTGTCTG-1,3464,367,K562,8.150179,0.105946882
s1d1_GCGCTTGCAACAGCCT-1,s1d1_GCGCTTGCAACAGCCT-1,7028,249,K562,8.857657,0.035429710
s1d1_CCTAAAGGTTTACCGT-1,s1d1_CCTAAAGGTTTACCGT-1,3765,68,K562,8.233503,0.018061089
s1d1_CAGGCTGGTCTAACAG-1,s1d1_CAGGCTGGTCTAACAG-1,3205,57,K562,8.072467,0.017784711
s1d1_CGCTCCATCGCCTGTT-1,s1d1_CGCTCCATCGCCTGTT-1,10145,243,K562,9.224736,0.023952686
s1d1_CGATTTGCACTGGCTG-1,s1d1_CGATTTGCACTGGCTG-1,3281,38,K562,8.095904,0.011581835


RUN SCENT prediction

In [12]:
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        if (nonzero_m > 0.05 & nonzero_a > 0.05) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            bs = boot::boot(df2, assoc, R = 100, formula = formula, 
                stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # if (p0 < 0.1) {
            #     bs = boot::boot(df2, assoc, R = 500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.05) {
            #     bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
            #     bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        }
    }
    object@SCENT.result <- res
    return(object)
}

In [13]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [14]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G.filter2)))
chr.run = chr.run[!chr.run %in% chr.done]

In [ ]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr.res = pairs.E2G.filter2[seqnames(pairs.E2G.filter2) == chr.tmp]
  gene_peak.chr = as.data.frame(mcols(pairs.E2G.chr.res)[,c("TargetGene","PeakName")])
  list.gene_peak.chr <- split(gene_peak.chr, seq_len(n.cores))
  
  list.obj.SCENT.chr <- lapply(list.gene_peak.chr, function(gene_peak.tmp) {
    obj.SCENT.tmp <- CreateSCENTObj(
      rna = matrix.rna[rownames(matrix.rna) %in% gene_peak.tmp$TargetGene,],
      atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
      meta.data = meta.data,
      peak.info = gene_peak.tmp,
      covariates = c("log.nUMI","percent.mito"), 
      celltypes = "celltype"
    )
  })
  
  cl <- makeCluster(n.cores)
  clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
  list.SCENT.result.chr <- parLapply(cl, list.obj.SCENT.chr, function(obj.SCENT.tmp) {
    obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
    return(obj.SCENT.tmp@SCENT.result)
  })
  stopCluster(cl)
  
  
  df.SCENT.result.chr = do.call(rbind,list.SCENT.result.chr)
  rownames(df.SCENT.result.chr) = paste(df.SCENT.result.chr$peak,
                                        df.SCENT.result.chr$gene,
                                        sep = "_")

  pairs.E2G.chr.res = pairs.E2G.chr.res[pairs.E2G.chr.res$PairName %in% rownames(df.SCENT.result.chr)]
  mcols(pairs.E2G.chr.res)[,c("beta","se","z","p","boot_basic_p")] = 
    df.SCENT.result.chr[pairs.E2G.chr.res$PairName,c("beta","se","z","p","boot_basic_p")]
  pairs.E2G.chr.res$boot_basic_p.log10 = -log10(pairs.E2G.chr.res$boot_basic_p)
  pairs.E2G.chr.res$sign = 1
  pairs.E2G.chr.res$sign[pairs.E2G.chr.res$beta < 0] = -1
  pairs.E2G.chr.res$boot_basic_p.log10.signed = pairs.E2G.chr.res$boot_basic_p.log10 * pairs.E2G.chr.res$sign
  saveRDS(pairs.E2G.chr.res,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.res)
}

[1] "chr1"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr10"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr11"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


Merge results

In [22]:
chr.merge = as.character(unique(seqnames(pairs.E2G.filter2)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [23]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 514720 ranges and 11 metadata columns:
       seqnames              ranges strand |  TargetGene               PeakName
          <Rle>           <IRanges>  <Rle> | <character>            <character>
  chr1     chr1 100037588-100039249      * |         AGL chr1-100037588-10003..
  chr1     chr1 100037588-100039249      * |      CDC14A chr1-100037588-10003..
  chr1     chr1 100037588-100039249      * |         DBT chr1-100037588-10003..
  chr1     chr1 100037588-100039249      * |        RTCA chr1-100037588-10003..
  chr1     chr1 100037588-100039249      * |       SASS6 chr1-100037588-10003..
   ...      ...                 ...    ... .         ...                    ...
  chrX     chrX       989889-990634      * |       P2RY8     chrX-989889-990634
  chrX     chrX       989889-990634      * |      PLCXD1     chrX-989889-990634
  chrX     chrX       989889-990634      * |     SLC25A6     chrX-989889-990634
  chrX     chrX     9940378-9941046      * |       TBL1X   ch

In [25]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100037588,100039249,AGL,K562,0.126753805,0.12614040,1.00486291,0.314962868,0.42,0.37675071,1,0.37675071
chr1,100037588,100039249,CDC14A,K562,0.129986263,0.04021798,3.23204334,0.001229084,0.02,1.69897000,1,1.69897000
chr1,100037588,100039249,DBT,K562,-0.330127450,0.15737234,-2.09774760,0.035927448,0.14,0.85387196,-1,-0.85387196
chr1,100037588,100039249,RTCA,K562,0.007859760,0.10734547,0.07321930,0.941631610,0.84,0.07572071,1,0.07572071
chr1,100037588,100039249,SASS6,K562,0.180676775,0.16027803,1.12727101,0.259627915,0.32,0.49485002,1,0.49485002
chr1,100037588,100039249,SLC30A7,K562,-0.067263629,0.07480803,-0.89914988,0.368572830,0.50,0.30103000,-1,-0.30103000
chr1,100037588,100039249,SLC35A3,K562,-0.013670576,0.13462625,-0.10154465,0.919118114,0.96,0.01772877,-1,-0.01772877
chr1,100037588,100039249,TRMT13,K562,-0.038708774,0.12488651,-0.30995161,0.756597757,0.82,0.08618615,-1,-0.08618615
chr1,100046477,100047244,AGL,K562,0.006814909,0.24475241,0.02784409,0.977786498,0.94,0.02687215,1,0.02687215


In [26]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
[1] Matrix_1.6-5         GenomicRanges_1.54.1 GenomeInfoDb_1.38.1 
[4] IRanges_2.36.0       S4Vectors_0.40.2     BiocGeneri